In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install transformers

In [ ]:
pip install SentencePiece

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
import networkx as nx

# Define your MHGRN model
class MHGRN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MHGRN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, adj_matrix):
        # Move the adj_matrix tensor to the same device as x
        adj_matrix = torch.tensor(adj_matrix, dtype=torch.float, device=x.device)

        x = torch.matmul(adj_matrix, x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

# Load your data from a .npy file
data = np.load("/content/drive/MyDrive/BrainTeaser/word_puzzle.npy", allow_pickle=True)[()]
questions = [entry['question'] for entry in data]
options = [entry['choice_list'] for entry in data]
correct_indices = [entry['label'] for entry in data]

# Create a directed graph
graph = nx.DiGraph()

# Create node features for questions
node_features = []  # You should replace this with meaningful features

# Create a set of nodes that are already added to the graph
added_nodes = set()

for entry in data:
    question = entry['question']
    choice_list = entry['choice_list']
    label = entry['label']

    # Add nodes for questions if they haven't been added
    if question not in added_nodes:
        graph.add_node(question)
        added_nodes.add(question)


    question_features = np.random.rand(64)
    node_features.append(question_features)

    # Connect the question node to choice nodes (edges represent relationships)
    for choice in choice_list:
        graph.add_edge(question, choice)
        added_nodes.add(choice)

# Create a list of node features in the order of nodes in the graph
node_features_list = [node_features[questions.index(node)] if node in questions else np.zeros(64) for node in graph.nodes()]


adj_matrix = np.array(nx.adjacency_matrix(graph).todense())

input_dim = len(node_features[0])
hidden_dim = 128
output_dim = 4


correct_indices = [correct_indices[questions.index(node)] if node in questions else 0 for node in graph.nodes()]

# Create TensorDatasets
train_features, val_features, train_labels, val_labels = train_test_split(node_features_list, correct_indices, test_size=0.2)

train_dataset = TensorDataset(torch.tensor(train_features, dtype=torch.float), torch.tensor(train_labels, dtype=torch.long))
train_loader = DataLoader(train_dataset, batch_size=1495, shuffle=True)

val_dataset = TensorDataset(torch.tensor(val_features, dtype=torch.float), torch.tensor(val_labels, dtype=torch.long))
val_loader = DataLoader(val_dataset, batch_size=1495, shuffle=False)


model = MHGRN(input_dim, hidden_dim, output_dim)
model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, batch in enumerate(train_loader, 0):
        inputs, labels = batch[0].to(device), batch[1].to(device)

        optimizer.zero_grad()
        outputs = model(inputs, adj_matrix)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {running_loss / (i + 1)}")



In [ ]:
# Set the model to evaluation mode
model.eval()

correct_predictions = 0
total_samples = 0

# Iterate through the validation dataset
with torch.no_grad():  # Disable gradient tracking during validation
    for batch in val_loader:
        inputs, labels = batch[0].to(device), batch[1].to(device)

        # Forward pass
        outputs = model(inputs)
        _, predicted = torch.max(outputs, dim=1)  # Get the predicted labels

        # Update counts
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

# Calculate accuracy
accuracy = (correct_predictions / total_samples) * 100.0
print(f'Validation Accuracy: {accuracy:.2f}%')
